In [21]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [295]:
class Algebra(object):
    def __init__(self, p, q):
        self._p = p
        self._q = q
        self._dim = p + q
        self._basis_blades_ints = [i for i in range(2 ** self._dim)]
        self._basis_blades_strs = [self._basis_blade_repr(i) for i in self._basis_blades_ints]
        self._basis_blades = self._create_basis_blades_dict()
        
    def __str__(self):
        return 'G_{{{},{}}}'.format(self._p, self._q)
    
    @property
    def p(self):
        return self._p
    
    @property
    def q(self):
        return self._p
    
    @property
    def dim(self):
        return self._dim
    
    def to_string(self, idx):
        return self._basis_blades_strs[idx]
    
    
    
    def __getitem__(self, name):
        return self._basis_blades[name]
    
    def _create_basis_blades_dict(self):
        
        basis_blades = {}
        for blade_int in self._basis_blades_ints:
            basis_blades[self._basis_blade_repr(blade_int)] = blade_int
        return basis_blades
    
    def _basis_blade_repr(self, blade_int):
        blade_bin = np.binary_repr(blade_int)[::-1]
        blade = ''
        for i in range(len(blade_bin)):
            if blade_bin[i] == '0':
                continue
            else:
                blade = blade + str(i+1)
        if blade == '':
            return str(1)
        else:
            return 'e' + blade
                               
class ConformalGeometricAlgebra(Algebra):
    def __init__(self):
        Algebra.__init__(self, 4, 1)

In [357]:
class Blade(object):
    def __init__(self, algebra, blade_name, scale=0.0):
        self._algebra = algebra
        if isinstance(blade_name, str):
            self._bitmap = self._algebra[blade_name]
        else: 
            self._bitmap = blade_name
        self._scale = scale
        self._grade = bitcount(self._bitmap)
    
    def __str__(self):
        return '{} * {}'.format(self._scale, self._algebra.to_string(self._bitmap))
    
    def __xor__(self, other):
        ''' Outer product '''
        if np.bitwise_and(self._bitmap, other._bitmap) != 0:
            return Blade(self._algebra, '1')
        else:
            return self.__mul__(other)
    
    def __mul__(self, other):
        ''' Geometric product '''
        return Blade(self._algebra, np.bitwise_xor(self._bitmap, other._bitmap),
                     self._scale * other._scale * sign(self._bitmap, other._bitmap))
    
    def __add__(self, other):
        return Multivector(self._algebra, [self, other])
        
    @property
    def grade(self):
        return self._bitmap
    
    def reverse(self):
        sign = np.power(-1, (self._grade * self._grade - 1) / 2) 
        return Blade(self._algebra, self._bitmap, scale = sign * self._scale)
        
        
        

In [359]:
cga = ConformalGeometricAlgebra()
a = Blade(cga, 'e3', 1.0)
b = Blade(cga, 'e3', 1.0)
a + b.reverse()

In [426]:
class Multivector(object):
    def __init__(self, algebra, blades):
        self._algebra = algebra
        self._blades = self._simplify(blades)
    
    def __str__(self):
        return str([b.__str__() for b in self._blades])
        
    def __xor__(self, other):
        raise NotImplementedError
        
    def __mul__(self, other):
        res = []
        for lhs in self._blades:
            for rhs in other._blades:
                res.append(lhs * rhs)
        return Multivector(self._algebra, res)
        
    def __add_(self, other):
        
        if isinstance(other, Blade):
            return Multivector(self._algebra. self._blades + [other])
        
    def reverse(self):
        return Multivector(self._algebra,[b.reverse() for b in self._blades])
    
    def _remove_nulls(self, blades):
        res = []
        for b in blades:
            if np.isclose(b._scale, 0.0):
                continue
            else:
                res.append(b)
        return res
    
    def _simplify(self, blades):
        prev = None
        for b in blades:
            if prev is None:
                prev = b
                continue
            elif prev._bitmap == b.bitmap:
                prev._scale += s
        res = []
        tmp = self._remove_nulls(blades)
        for i in range(len(tmp)-1):
            print tmp[i]
            print tmp[i+1]
            if tmp[i]._bitmap == tmp[i+1]._bitmap:
                res.append(Blade(self._algebra, tmp[i]._bitmap, tmp[i]._scale + tmp[i+1]._scale))
            else:
                res.append(tmp[i])
                
        return self._simplify(res)
                
        


In [427]:
R = Multivector(cga, [Blade(cga, '1', cos(pi/4)),
                 Blade(cga, 'e12', -sin(pi/4)), Blade(cga, 'e13', 0) , Blade(cga, 'e23', 0)])
# print(R)

# a = Multivector(cga, [Blade(cga, 'e1', 1.0)]) 

Ra = R * a
Ra._blades
# print b

0.707106781187 * 1
-0.707106781187 * e12


RuntimeError: maximum recursion depth exceeded

In [236]:
cga = ConformalGeometricAlgebra()
e1 = Multivector(cga, ['1', 'e12', 'e13', 'e23'])
print(e1)
cga._basis_blade_repr(31)

[0, 3, 5, 6]


'e12345'

In [208]:
def bitcount(a):
    a -= np.bitwise_and(np.right_shift(a, 1), 0x55555555)
    a  = np.bitwise_and(a, 0x33333333) + np.bitwise_and(np.right_shift(a, 2), 0x33333333)
    a = np.bitwise_and(a + np.right_shift(a, 4), 0xF0F0F0F)
    a += np.right_shift(a, 8)
    a += np.right_shift(a, 16)
    return np.bitwise_and(a, 0x0000003F)

def sign(a, b):
    a = np.right_shift(a, 1)
    sum_ = 0
    while (a != 0):
        sum_ += bitcount(np.bitwise_and(a,b))
        a = np.right_shift(a, 1)
    return 1.0 if np.bitwise_and(sum_, 1) == 0 else -1.0


def geometric_product(a_w, a, b_w, b):
    return (a_w * b_w * sign(a,b), (a ^ b))
geometric_product(2.0, cga['e3'], 3.0,  cga['e1'])

(-6.0, 5)

In [198]:
e3 = cga['e3']
e1 = cga['e1']
sign(e3, e1)

-1.0

In [188]:
np.bitwise_and(3,1)

1

In [ ]:
a -= np.bitwise_and(np.right_shift(a, 1), 0x55555555)
    a = np.bitwise_and(a, 0x33333333) + np.bitwise_and(np.right_shift(a, 2), 0x33333333)
    return np.right_shift(np.bitwise_and((np.bitwise_and(a + np.right_shift(a, 4), 0xF0F0F0F) * 0x1010101), 0xffffffff), 24)
